In [1]:
from utils_vae import read_field
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Model

# import keras 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse
# import sys, importlib

# from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import LearningRateScheduler
# from tensorflow.keras.utils import plot_model
import tensorflow as tf

from tensorflow.keras import regularizers
from tensorflow.keras.layers import Input, Dense,Flatten,Concatenate ,Conv1D
from VAES import dense_gen,cloud_model,schedule

from tensorflow.keras import regularizers
from tensorflow.keras import initializers

2022-12-02 13:53:42.150984: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /glade/u/apps/ch/opt/mpt_fmods/2.25/intel/19.1.1:/glade/u/apps/ch/opt/mpt/2.25/lib:/glade/u/apps/opt/intel/2020u1/compilers_and_libraries/linux/lib/intel64:/glade/u/apps/ch/os/usr/lib64:/glade/u/apps/ch/os/usr/lib:/glade/u/apps/ch/os/lib64:/glade/u/apps/ch/os/lib
2022-12-02 13:53:42.151023: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
path = "/glade/scratch/sshamekh/dyamond/SAM_highres/"

# %%time
import sys, importlib
importlib.reload(sys.modules['utils_vae'])
from utils_vae import train_test_data,read_data_for_z
t1 = 0
t2 = 8
high_res = 4.0 # km 
large_scale = 128 #km
x_hr,x_lg,x_lg2, y_out = read_data_for_z(path, high_res,large_scale,t1,t2,threshold_precip = 0.01,mask_threshold = False)


Reading high resolution input
qvi
no scaling
(760, 24, 287, 32, 32, 1)
Reading large-scale inputs
qvi
ts
tas
huss
(760, 24, 287, 4)
Reading large-scale inputs2
vas
uas
hfss
hfls
(760, 24, 287, 4)
Reading large-scale outputs
pracc
(760, 24, 287, 1)
[[[[0.8208742125920748]]]]


In [3]:
pw_anomaly = x_hr - np.mean(x_hr,axis = (3,4),keepdims=True)
x_lg = np.concatenate((x_lg,x_lg2[:,:,:,2:]),axis = -1)

In [ ]:
def make_z(hrdata,lgdata):
    path = '/glade/u/home/sshamekh/results/dyamond/4nodes/'
    encoder_result= keras.models.load_model(path + 'encoder_4_32_mse_anomaly')
    precip_nn =  keras.models.load_model(path + 'precip_dense_vae_4_32_mse_anomaly')
    
    tt,lat,lon,nx,ny,ff = hrdata.shape
    hrdata = hrdata.reshape(tt*lat*lon,nx,ny,ff)
    lgd = lgdata.reshape(tt*lat*lon,-1)
    
    zall = encoder_result.predict(hrdata)
    innew = np.concatenate((zall,lgd),axis = -1)
    precip = precip_nn.predict(innew)
    zall = zall.reshape(tt,lat,lon,zall.shape[-1])
    return zall,precip

z_true,precip = make_z(pw_anomaly*2,x_lg)
    

2022-12-02 14:10:15.881583: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /glade/u/apps/ch/opt/mpt_fmods/2.25/intel/19.1.1:/glade/u/apps/ch/opt/mpt/2.25/lib:/glade/u/apps/opt/intel/2020u1/compilers_and_libraries/linux/lib/intel64:/glade/u/apps/ch/os/usr/lib64:/glade/u/apps/ch/os/usr/lib:/glade/u/apps/ch/os/lib64:/glade/u/apps/ch/os/lib
2022-12-02 14:10:15.881640: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-02 14:10:15.881678: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (r6i6n31): /proc/driver/nvidia/version does not exist
2022-12-02 14:10:15.933218: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use th

126138/163590 [======================>.......] - ETA: 31s

In [8]:
def rand(nn):
    import random
    random.seed(9001)
    x = [random.randint(1,100) for i in range(0,nn)]
    return np.array(x)
def chunk(data,window):
    for i in range(0,window,2):
        datain = np.copy(data[i:,:])
        nt,nlat,nlon,nfeature = datain.shape
        nchunk = int(nt/window)
        dtin = datain[:nchunk*window ,:,:,:]
        dtin = np.reshape(dtin,(nchunk,window,nlat,nlon,nfeature))
        if i==0: 
            data_all = np.copy(dtin)
        else: 
            data_all = np.concatenate((data_all,dtin),axis = 0 )
    print (data_all.shape,'data_all')
    return data_all



def chunk_horizontal(data,window):
    for i in range(window):
        for j in range(window):
            datain = np.copy(data[:,:,i:,j:])
            nchunk,nw,nlat,nlon,nfeature = datain.shape
            nch_lat = int(nlat/window)
            nch_lon = int(nlon/window)
            dtin = datain[:,:,:nch_lat*window,:nch_lon*window,:]
            dtin = np.reshape(dtin,(nchunk,nw,nch_lat,window,nch_lon,window,nfeature))
            print (dtin.shape)
            dtin = np.moveaxis(dtin,3,4 )
            if i==0 and j==0: 
                data_all = np.copy(dtin)
            else: 
                data_all = np.concatenate((data_all,dtin),axis = 0 )
        return data_all
def reshape_data(data):
    nt,window,nlat,nlon,nfeature = data.shape
    
    dtin = np.moveaxis(data,1,3)
    print (dtin.shape)
    dtin = np.reshape(dtin,(nlat*nlon*nt,window,nfeature))
    return dtin

timeframe=3


x_all =  chunk(x_lg,timeframe)
y_all =  chunk(y_out,timeframe)
zz_all = chunk(z_true,timeframe)

mask =  rand(y_all.shape[0]) < 70
in_train,in_test = x_all[mask,:,:,:], x_all[~mask,:,:,:]
out_train,out_test = y_all[mask,:,:,:], y_all[~mask,:,:,:]
zz_train,zz_test = zz_all[mask,:,:,:], zz_all[~mask,:,:,:]

x_test  = reshape_data (in_test)
y_train = reshape_data(out_train)
y_test = reshape_data(out_test)

zz_train = reshape_data(zz_train)
zz_test = reshape_data(zz_test)


(505, 3, 24, 287, 6) data_all
(505, 3, 24, 287, 1) data_all
(505, 3, 24, 287, 4) data_all
(329, 24, 287, 3, 6)
(176, 24, 287, 3, 6)
(329, 24, 287, 3, 1)
(176, 24, 287, 3, 1)
(329, 24, 287, 3, 4)
(176, 24, 287, 3, 4)


In [9]:
for i in range (timeframe):
    index = np.where(y_train[:,i,0]>.010)[0]
    y_train = y_train[index,:]
    x_train = x_train[index,:]
    zz_train = zz_train[index,:]
   

    index = np.where(y_test[:,-1,0]>.010)[0]
    y_test = y_test[index,:]
    x_test = x_test[index,:]
    zz_test = zz_test[index,:]


In [10]:
mmzz = (np.mean(np.concatenate((zz_test[:,-1,:],zz_train[:,-1,:]),axis = 0),axis = 0,keepdims=True)).reshape(1,1,4)
zzstd = (np.std(np.concatenate((zz_test[:,-1,:],zz_train[:,-1,:]),axis = 0),axis = 0,keepdims=True) ).reshape(1,1,4)



In [11]:
zz_train = (zz_train - mmzz) / zzstd
zz_test = (zz_test - mmzz) / zzstd

In [31]:
units = [256,256,128,64]
l1 = 0e-7

initializer = None
batch_size = 128
epochs = 100
activation = tf.nn.relu
output_shape = 4
inshapeX = x_train[0,2:,:].shape
inzshape = zz_train[0,:2,:].shape

val_split = 0.2


input_lg = Input(shape=(inshapeX), name='inputs_lg')
in_lg = Flatten()(input_lg)
input_z = Input(shape=(inzshape), name='inputs_z')
in_z = Flatten()(input_z)


intotal = Concatenate(axis  = 1)([in_lg,in_z])

xs =  Dense(units=units[0],activation = None,kernel_initializer=initializer,
            name = 'dense1')(intotal)

xs =  Dense(units=units[1],activation = activation,kernel_initializer=initializer,
           name = 'dense2')(xs)

xs =  Dense(units=units[2],activation = activation,kernel_initializer=initializer,
           name = 'dense3')(xs)

xs =  Dense(units=units[3],activation = activation,kernel_initializer=initializer,
            name = 'dense4')(xs)

y_predict =  Dense(units=output_shape,activation = None,name = 'prediction')(xs)

nn_precip = Model(inputs = [input_lg,input_z],
              outputs = [y_predict]) 

opt = keras.optimizers.Adam(lr=0.0001)

nn_precip.compile(optimizer = opt,loss = 'mse')

earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto')
callback_lr=LearningRateScheduler(schedule,verbose=1)


hist = nn_precip.fit({'inputs_lg' : x_train[:,2:,:],
                      'inputs_z' : zz_train[:,:2,:],
                   
                      },
                  {'prediction': zz_train[:,2,:]},
                   epochs = epochs,  validation_split = val_split,callbacks= [earlyStopping,callback_lr],
                     batch_size = batch_size,verbose = 1, shuffle=True)


Epoch 1: LearningRateScheduler setting learning rate to 0.0004.
Epoch 1/100
3000/3000 [==============================] - 5s 2ms/step - loss: 0.0469 - val_loss: 0.0450 - lr: 4.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 0.0004.
Epoch 2/100
3000/3000 [==============================] - 4s 1ms/step - loss: 0.0366 - val_loss: 0.0482 - lr: 4.0000e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.0004.
Epoch 3/100
3000/3000 [==============================] - 4s 1ms/step - loss: 0.0365 - val_loss: 0.0455 - lr: 4.0000e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.0004.
Epoch 4/100
3000/3000 [==============================] - 4s 1ms/step - loss: 0.0366 - val_loss: 0.0449 - lr: 4.0000e-04

Epoch 5: LearningRateScheduler setting learning rate to 0.0004.
Epoch 5/100
3000/3000 [==============================] - 4s 1ms/step - loss: 0.0365 - val_loss: 0.0457 - lr: 4.0000e-04

Epoch 6: LearningRateScheduler setting learning rate to 0.0004.
Epoch 6/10

In [26]:
zz_predict= nn_precip.predict([x_test[:,2:,:],zz_test[:,1:2,:]])

14997/14997 [==============================] - 11s 725us/step


In [28]:
from sklearn.metrics import r2_score
r2s = [r2_score(zz_test[:,2,0],zz_predict[:,0]),r2_score(zz_test[:,-1,1],zz_predict[:,1]),r2_score(zz_test[:,-1,2],zz_predict[:,2]),r2_score(zz_test[:,-1,3],zz_predict[:,3])]

In [29]:
r2s

([0.9366492987966142,
  0.9735186645212718,
  0.9661567450141424,
  0.9567288298220242],)

In [169]:
def scatter_hist(x, y, ax, ax_histx, ax_histy,i,r2):
    # no labels
    ax_histx.tick_params(axis="x", labelbottom=False)
    ax_histy.tick_params(axis="y", labelleft=False)
    
    ax.tick_params(axis="both", labelsize= 16)

    ax_histy.set_xticks([])
    # ax_histx.set_xticks([])
    # ax_histy.set_yticks([])
    ax_histx.set_yticks([])
    ax_histx.spines['top'].set_visible(False)
    ax_histx.spines['right'].set_visible(False)
    ax_histx.spines['bottom'].set_visible(False)
    ax_histx.spines['left'].set_visible(False)
    ax_histy.spines['top'].set_visible(False)
    ax_histy.spines['right'].set_visible(False)
    ax_histy.spines['bottom'].set_visible(False)
    ax_histy.spines['left'].set_visible(False)
    ax.set_xlabel(r'$OrgT_{%s}$'%str(i),fontsize = 18)
    ax.set_ylabel(r'$OrgP_{%s}$'%str(i),fontsize = 18)
    # the scatter plot:
    ax.scatter(x, y)
    ax.text(-1.2,1.2,r'$R^2 = %2.2f$'%r2, fontsize= 18)

    # now determine nice limits by hand:
    binwidth = 0.01
    xymax = max(np.max(np.abs(x)), np.max(np.abs(y)))
    lim = (int(xymax/binwidth) + 1) * binwidth

    bins = np.arange(-lim, lim + binwidth, binwidth)
    # bins = 100
    ax_histx.hist(x, bins=bins)
    ax_histy.hist(y, bins=bins, orientation='horizontal')
    ax.set_ylim(-1.5,1.5)
    ax.set_xlim(-1.5,1.5)
    
def scatter_hist_2(x, y, ax, i,r2):
    # no labels
    ax_histx = ax.twinx()
    ax_histy = ax.twiny()
    ax_histx.tick_params(axis="x", labelbottom=False)
    ax_histy.tick_params(axis="y", labelleft=False)
    
    ax.tick_params(axis="both", labelsize= 16)

    ax_histy.set_xticks([])
    # ax_histx.set_xticks([])
    # ax_histy.set_yticks([])
    ax_histx.set_yticks([])
    ax_histx.spines['top'].set_visible(False)
    ax_histx.spines['right'].set_visible(False)
    ax_histx.spines['bottom'].set_visible(False)
    ax_histx.spines['left'].set_visible(False)
    ax_histy.spines['top'].set_visible(False)
    ax_histy.spines['right'].set_visible(False)
    ax_histy.spines['bottom'].set_visible(False)
    ax_histy.spines['left'].set_visible(False)
    ax.set_xlabel(r'$OrgT_{%s}$'%str(i),fontsize = 18)
    ax.set_ylabel(r'$OrgP_{%s}$'%str(i),fontsize = 18)
    # the scatter plot:
    ax.scatter(x[::6], y[::6],color = 'steelblue',alpha = 0.6,s = 60, facecolors='none')
    ax.text(-.8,.8,r'$R^2 = %2.2f$'%r2, fontsize= 18)
    ax.set_yticks(ticks=[-1,-0.5,0,0.5,1])
    # now determine nice limits by hand:
    binwidth = 0.01
    xymax = max(np.max(np.abs(x)), np.max(np.abs(y)))
    lim = (int(xymax/binwidth) + 1) * binwidth

    bins = np.arange(-lim, lim + binwidth, binwidth)
    # bins = 100
    ax_histx.hist(x, bins=bins,color = 'olivedrab',alpha = 0.8)
    ax_histy.hist(y, bins=bins, orientation='horizontal',color = 'firebrick',alpha = 0.8)
    ax.set_ylim(-1.,1.)
    ax.set_xlim(-1.,1.)
    ax_histy.set_xlim(0,120000)
    ax_histx.set_ylim(0,120000)